In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from simpletransformers.classification import ClassificationModel, ClassificationArgs

c:\tesla-news-classifier\env\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
c:\tesla-news-classifier\env\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [11]:
datapath = 'data/'
train_df = pd.read_csv(datapath + 'train.csv')
test_df = pd.read_csv(datapath + 'test.csv')

In [3]:
train_df = train_df[["title", "labels"]]
train_df.columns = ["text", "labels"]

test_df = test_df[["title", "labels"]]
test_df.columns = ["text", "labels"]

In [4]:
# model_name = "bert-base-cased"
# model_type = "bert"

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
model_type = "roberta"

model_args = ClassificationArgs()
model_args.no_cache = True
model_args.fp16 = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.output_dir = f"outputs/{model_type}"
model_args.best_model_dir = f"outputs/{model_type}/best_model"
model_args.evaluate_during_training = True
#model_args.lazy_loading = True
#model_args.sliding_window = True
model_args.max_seq_length = 128
model_args.num_train_epochs = 20
model_args.train_batch_size = 64
model_args.eval_batch_size = 32

In [5]:
# Calculate the label weight distribution, small label lize give more weight
distribution = train_df.labels.value_counts(normalize=False)
print(distribution)
dist = distribution.max() / distribution
print(dist)
dist = dist.sort_index().tolist()
print(dist)


0    185
2     83
1     61
Name: labels, dtype: int64
0    1.000000
2    2.228916
1    3.032787
Name: labels, dtype: float64
[1.0, 3.0327868852459017, 2.2289156626506026]


In [6]:
model = ClassificationModel(model_type, model_name, num_labels=3, weight=dist, args=model_args, use_cuda=True)
#model = ClassificationModel(model_type, model_name, num_labels=3, args=model_args, use_cuda=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.train_model(train_df, eval_df=test_df)

c:\tesla-news-classifier\env\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


c:\tesla-news-classifier\env\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(120,
 {'global_step': [6,
   12,
   18,
   24,
   30,
   36,
   42,
   48,
   54,
   60,
   66,
   72,
   78,
   84,
   90,
   96,
   102,
   108,
   114,
   120],
  'mcc': [0.07290662074192468,
   0.006547533639309213,
   0.0,
   0.20820787755542516,
   0.13933939873256457,
   0.21409148755894863,
   0.24769467506527604,
   0.24237169129417116,
   0.23962893381255243,
   0.2680112385532181,
   0.2739092183847996,
   0.22439323750049633,
   0.2713718559335609,
   0.27834975179071003,
   0.23531083140854456,
   0.30525582212173497,
   0.30525582212173497,
   0.28148265170777986,
   0.3192538085768905,
   0.3192538085768905],
  'train_loss': [2.1652233600616455,
   1.1153632402420044,
   1.0994154214859009,
   1.0545334815979004,
   1.4076457023620605,
   0.7545440793037415,
   0.6192498207092285,
   0.4273402988910675,
   0.05959172919392586,
   0.1271858811378479,
   0.06117497757077217,
   0.029655523598194122,
   0.017472099512815475,
   0.012410993687808514,
   0.010797496885061264

In [10]:
#preds, out = model.predict(["consumer reports says it can't recommend the tesla model 3"])
#preds, out = model.predict(["tesla denies safety problems with model s suspensions"])
preds, out = model.predict(["us ends probe of tesla fatal crash without seeking recall"])
print(preds)
print(out)


[2]
[[-1.97070312 -2.08398438  4.75      ]]


In [ ]:
train_df.text.str.split(' ').str.len()